# CREATING FACT TABLE

Reading Silver Data

In [0]:
%sql
SELECT * FROM parquet.`abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales`

In [0]:
df_silver = spark.sql("""
                      SELECT * FROM parquet.`abfss://silver@carprojectdatalakejess.dfs.core.windows.net/car_sales`
""")

In [0]:
df_silver.display()

Reading All The DIMS Created

In [0]:
df_dealer = spark.sql("SELECT * FROM cars_catalog.gold.dim_dealer")


df_branch = spark.sql(" SELECT * FROM cars_catalog.gold.dim_branch")

df_model = spark.sql(" SELECT * FROM cars_catalog.gold.dim_model")

df_date = spark.sql(" SELECT * FROM cars_catalog.gold.dim_date")




Joining Our Keys and bringing them to the Fact Table

In [0]:
df_fact = df_silver.join(df_branch, df_silver["Branch_ID"]== df_branch["Branch_ID"], "left")\
                   .join(df_dealer, df_silver["Dealer_ID"]== df_dealer["Dealer_ID"], "left")\
                   .join(df_model, df_silver["Model_ID"]== df_model["Model_ID"], "left")\
                   .join(df_date, df_silver["DATE_ID"]== df_date["DATE_ID"], "left")\
                   .select(df_silver["Revenue"], df_silver["Units_Sold"], df_silver["RevPerUnit"],df_branch["dim_branch_key"], df_model["dim_model_key"], df_dealer["dim_dealer_key"], df_date["dim_date_key"])



In [0]:
df_fact.display()

# Writing Fact Table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.factsales"):
    deltatb = DeltaTable.forName(spark, "cars_catalog.gold.factsales")
    deltatb.alias("t") \
        .merge(
            df_fact.alias("s"),
            """
            t.dim_branch_key = s.dim_branch_key AND
            t.dim_dealer_key = s.dim_dealer_key AND
            t.dim_model_key = s.dim_model_key AND
            t.dim_date_key = s.dim_date_key
            """
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    df_fact.write.format("delta") \
        .mode("overwrite") \
        .option("path", "abfss://gold@carprojectdatalakejess.dfs.core.windows.net/factsales") \
        .saveAsTable("cars_catalog.gold.factsales")


In [0]:
%sql
select * from cars_catalog.gold.factsales